In [1]:
# basics
import numpy as np
import pandas as pd
import datetime
import re

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);

np.seterr(divide = 'ignore');


In [2]:
# load data into datawrapper classes
measure_data = mc.COVID19_measures(download_data = False, measure_level = 2, only_first_dates = True, expand_measure_names = True)
jhu_data     = cdc.CoronaData(download_data = False)

# remove and rename countries to match the JFU database and the measures database
measure_data.RemoveCountry('Diamond Princess')
measure_data.RenameCountry('France (metropole)', 'France')
measure_data.RenameCountry('South Korea', 'Korea, South')
measure_data.RenameCountry('Czech Republic', 'Czechia')


In [3]:
# helper functions

def date2vector(implementdate, start = '22/1/20', end = None, shiftdays = 0):
    # generate vector of 0s and 1s when measure is implemented or not
    starttime     = datetime.datetime.strptime(start,         '%d/%m/%y')
    if end is None:
        endtime   = datetime.datetime.today()
    else:
        endtime   = datetime.datetime.strptime(end,           '%d/%m/%y')
    implementtime = datetime.datetime.strptime(implementdate, '%d/%m/%Y')
    
    totaldays   = (endtime       - starttime).days
    measuredays = (implementtime - starttime).days
    
    vec         = np.zeros(totaldays)
    vec[min(measuredays+shiftdays,len(vec)-1):] = 1
    
    return vec


def ConvertDateFormat(date):
    m,d,y = date.split('/')
    return '{:02d}/{:02d}/{:02d}'.format(int(d),int(m),int(y))


def CleanUpMeasureName(measurename):
    # regression model formula can't contain special characters
    return ''.join([mn.capitalize() for mn in measurename.replace(',','').replace('-','').replace('/','').split(' ')])


def GetMeasureIDs(countrylist = None, measure_level = 2, mincount = None, extend_measure_names = False):
    if countrylist is None:
        countrylist = measure_data.countrylist # use ALL countries
    
    measurelist = {}
    
    # get all restrictions from countries
    for country in countrylist:
        country_measures = measure_data.CountryData(country, measure_level = 2, extend_measure_names = extend_measure_names)
        for measurename, initialdata in country_measures.items():
            if not measurename in measurelist.keys():
                measurelist[measurename] = 0
            measurelist[measurename] += 1
    
    if not mincount is None:
        # rebuild dict with restrictions
        measurelist = {k:v for k,v in measurelist.items() if v >= mincount}

    return measurelist


def SmoothTrajectories3(traj):
    if len(traj) > 3:
        newtraj       = np.zeros(len(traj))
        newtraj[0]    = (             2 * traj[0]    + traj[1] )/3.
        newtraj[1:-1] = (traj[0:-2] + 2 * traj[1:-1] + traj[2:])/4.
        newtraj[-1]   = (traj[-2]   + 2 * traj[-1]             )/3.
        return newtraj
    else:
        return traj


def GetCountryTrajectories(countrylist = None, data = 'Confirmed', startcases = None, maxlen = None, smooth = False):
    if countrylist is None:
        countrylist = jhu_data.countrylist
    
    trajectories = {}
    for country in [c for c in countrylist if c in jhu_data.countrylist]:
        ctraj = np.array(jhu_data.CountryData(country)[data], dtype = np.float)
        starttraj = 0
        if not startcases is None:
            starttraj = np.argmax(ctraj >= startcases)
            ctraj = ctraj[starttraj:]
            if not maxlen is None:
                ctraj = ctraj[:min(maxlen,len(ctraj))]
        trajectories[country] = {}
        if smooth:
            ctraj = SmoothTrajectories3(ctraj)
        trajectories[country]['traj'] = ctraj
        trajectories[country]['startdate'] = ConvertDateFormat(jhu_data.CountryData(country)['Date'][starttraj])
    
    return trajectories

        
def GetRegressionDF(countrylist = None, measure_level = 2, shiftdays = 0, verbose = False, maxlen = None, smooth = None):
    # construct pd.DataFrame used for regression
    
    # get trajectories and measure list for all countries in 'countrylist'
    trajectories         = GetCountryTrajectories(countrylist = countrylist, data = 'Confirmed', startcases = 30, maxlen = maxlen, smooth = smooth)
    measureIDs           = measure_data.MeasureList(countrylist = countrylist, measure_level = 2, mincount = 5)
    cleaned_measurelist  = {CleanUpMeasureName(mn):count for mn,count in measureIDs.items()}
    regressionDF         = None
    
    if verbose:
        print(measureIDs)
    
    for country in trajectories.keys():
        if country in measure_data.countrylist:

            # ********************************************
            # change observable to regress here:
            observable                  = np.diff(np.log(trajectories[country]['traj']))
            obslen                      = len(observable)
            # ********************************************
            
            DF_country = measure_data.ImplementationTable(country           = country,
                                                        measure_level     = 2,
                                                        startdate         = trajectories[country]['startdate'],
                                                        shiftdays         = shiftdays,
                                                        maxlen            = obslen,
                                                        clean_measurename = True)
            
            for measurename in DF_country.columns:
                if measurename not in measureIDs.keys():
                    DF_country.drop(labels = measurename, axis = 'columns')
            
            DF_country['Country']    = country
            DF_country['Observable'] = observable

            
            if not (np.isnan(DF_country['Observable']).any() or np.isinf(DF_country['Observable']).any()):

                if regressionDF is None:
                    regressionDF = DF_country
                else:
                    regressionDF = pd.concat([regressionDF,DF_country], ignore_index = True, sort = False)
    
    # not implemented measures should be NaN values, set them to 0
    regressionDF.fillna(0, inplace = True)
    
    return regressionDF, cleaned_measurelist



def GetCountryMasks(regrDF):
    countrylist = list(regrDF['Country'].unique())
    maskdict = {}
    for country in countrylist:
        mask = list(regrDF['Country'] == country)
        maskdict[country] = mask
    return maskdict



def CrossValidation(data = None, drop_cols = None, outputheader = None, alpha = 1e-5, alphacountry = None, crossvalcount = 10):
    # output df
    result_DF = None
    
    # assign samples to each of the crossvalidation chunks
    datalen = len(data)
    chunklen   = np.ones(crossvalcount,dtype = np.int) * (datalen // crossvalcount)
    chunklen[:datalen%crossvalcount] += 1
    samples    = np.random.permutation(
                    np.concatenate(
                        [i*np.ones(chunklen[i],dtype = np.int) for i in range(crossvalcount)]
                    ))
    
    # generate formula of model directly from columns in DataFrame
    measurelist = list(data.columns)
    measurelist.remove('Observable')
    measurelist.remove('Country')
    formula = 'Observable ~ C(Country) + ' + ' + '.join(measurelist)
    
    # iterate over all chunks
    for xv_index in range(crossvalcount):
        # generate training and test models
        trainidx = (samples != xv_index)
        testidx  = (samples == xv_index)
        trainmodel = smf.ols(formula = formula, data = data[trainidx], drop_cols = drop_cols)
        testmodel  = smf.ols(formula = formula, data = data[testidx],  drop_cols = drop_cols)
    
        # if no alphacountry value is given, assume same penalty (alpha) for all paramters
        if alphacountry is None:
            results = trainmodel.fit_regularized(alpha = alpha, L1_wt = 1)
        else:
            # otherwise, penalize measures and countries differently
            # no penality for the 'Intercept'
            alphavec = np.zeros(len(trainmodel.exog_names))
            for i,exogname in enumerate(trainmodel.exog_names):
                if exogname[:10] == 'C(Country)': alphavec[i] = alphacountry
                elif exogname == 'Intercept':     alphavec[i] = 0
                else:                             alphavec[i] = alpha
            results = trainmodel.fit_regularized(alpha = alphavec, L1_wt = 1)

        # generate list of test params
        # random sampling could have discarded some of these parameters in the test case
        test_params = []
        for paramname in testmodel.exog_names:
            if paramname in results.params.keys():
                test_params.append(results.params[paramname])
            else:
                test_params.append(0)

        obs_train   = np.array(trainmodel.endog)
        obs_test    = np.array(testmodel.endog)
        pred_train  = trainmodel.predict(results.params)
        pred_test   = testmodel.predict(test_params)
            
        # store results first in dict
        result_dict = {}
        if outputheader is not None:
            result_dict.update(outputheader)
        
        result_dict['Iteration']        = xv_index
        
        result_dict['Loglike Training'] = trainmodel.loglike(results.params)
        result_dict['Loglike Test']     = testmodel.loglike(np.array(test_params))

        result_dict['R2 Training']      = 1 - np.sum((obs_train - pred_train)**2)/np.sum((obs_train - np.mean(obs_train))**2)
        result_dict['R2 Test']          = 1 - np.sum((obs_test - pred_test)**2)/np.sum((obs_test - np.mean(obs_test))**2)

        result_dict['RSS Training']     = np.sum((obs_train - pred_train)**2)
        result_dict['RSS Test']         = np.sum((obs_test - pred_test)**2)

        result_dict.update({k:v for k,v in results.params.items()})
        
        # append dict to df
        if result_DF is None:
            result_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
        else:
            result_DF = result_DF.append(result_dict, ignore_index = True)
    return result_DF

In [4]:
# setup all dataframes with different shifts in the effect of measures

maxshift           = 10
regrDF             = {}

for shiftdays in range(0, maxshift + 1):
    print('shiftdays = {}'.format(shiftdays))
    
    regrDF[shiftdays], measurelist = GetRegressionDF( countrylist = measure_data.countrylist,
                                                      shiftdays   = shiftdays,
                                                      smooth      = True)
    


shiftdays = 0


/usr/lib64/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/lib64/python3.7/site-packages/numpy/lib/function_base.py:1273: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


shiftdays = 1
shiftdays = 2
shiftdays = 3
shiftdays = 4
shiftdays = 5
shiftdays = 6
shiftdays = 7
shiftdays = 8
shiftdays = 9
shiftdays = 10


In [14]:
for repetition in range(3):
    print('repetition = {}'.format(repetition))
    allres_smallshift = None

    for shiftdays in np.arange(5,10):
        print('  shiftdays = {}'.format(shiftdays))

        for alpha in np.power(10,np.linspace(-6, -3.4, num = 14, endpoint = True)):
            print('     alpha = {:.6f}, time = {}'.format(alpha, datetime.datetime.now().strftime('%H:%M:%S')))
            
            resultDF = CrossValidation( data         = regrDF[shiftdays],
                                        outputheader = {'shiftdays':shiftdays, 'alpha':alpha},
                                        alpha        = alpha)

            if allres_smallshift is None:
                allres_smallshift = resultDF
            else:
                allres_smallshift = pd.concat([allres_smallshift, resultDF])
    # store results in csv
    allres_smallshift.to_csv('data/results_largeshift_200413_{}.csv'.format(repetition))

repetition = 0
  shiftdays = 5
     alpha = 0.000001, time = 14:39:29
     alpha = 0.000002, time = 14:40:17
     alpha = 0.000003, time = 14:41:09
     alpha = 0.000004, time = 14:42:05
     alpha = 0.000006, time = 14:42:57
     alpha = 0.000010, time = 14:43:47
     alpha = 0.000016, time = 14:44:39
     alpha = 0.000025, time = 14:45:34
     alpha = 0.000040, time = 14:46:29
     alpha = 0.000063, time = 14:47:18
     alpha = 0.000100, time = 14:48:05
     alpha = 0.000158, time = 14:48:49
     alpha = 0.000251, time = 14:49:27
     alpha = 0.000398, time = 14:50:05
  shiftdays = 6
     alpha = 0.000001, time = 14:50:35
     alpha = 0.000002, time = 14:51:36
     alpha = 0.000003, time = 14:52:51
     alpha = 0.000004, time = 14:54:03
     alpha = 0.000006, time = 14:55:49
     alpha = 0.000010, time = 14:58:21
     alpha = 0.000016, time = 15:01:12
     alpha = 0.000025, time = 15:03:34
     alpha = 0.000040, time = 15:05:25
     alpha = 0.000063, time = 15:06:01
     alpha = 0.00

KeyboardInterrupt: 

In [71]:
allres_doublealpha = None
#for repetition in range(10):
for alpham in np.power(10.,np.linspace(-6,-3,num=4)):
    for alphac in np.power(10.,np.linspace(-6,-3,num=4)):
        print('{:.6f} {:.6f}'.format(alpham,alphac))
        result_df = CrossValidation( data         = regrDF40[7],
                                     outputheader = {'shiftdays':shiftdays, 'alpha':alpham, 'alphacountry':alphac},
                                     alpha        = alpham,
                                     alphacountry = alphac)
        #print(result_df)
        if allres_doublealpha is None:
            allres_doublealpha = result_df
        else:
            allres_doublealpha = pd.concat([allres_doublealpha, result_df])


0.000001 0.000001
0.000001 0.000010
0.000001 0.000100
0.000001 0.001000
0.000010 0.000001
0.000010 0.000010
0.000010 0.000100
0.000010 0.001000
0.000100 0.000001
0.000100 0.000010
0.000100 0.000100
0.000100 0.001000
0.001000 0.000001
0.001000 0.000010
0.001000 0.000100
0.001000 0.001000


In [72]:
allres_doublealpha.to_csv('results_doublealpha_200412_2.csv')